In [1]:
from Bio import Entrez
Entrez.email = 'lukas.becker@hhu.de'

In [43]:
handle = Entrez.esearch(db="assembly",term="Curvibacter",retmode="xml")
record = Entrez.read(handle)
handle.close()

In [44]:
record #281915 == Curvibacter

{'Count': '28', 'RetMax': '20', 'RetStart': '0', 'IdList': ['10501171', '10449501', '10109261', '7805401', '7793981', '7787691', '7781621', '7743831', '7742251', '7740621', '7670861', '7553411', '7524191', '7368221', '2316721', '2316651', '2191521', '1960111', '1940331', '1369251'], 'TranslationSet': [{'From': 'Curvibacter', 'To': '"Curvibacter"[Organism]'}], 'TranslationStack': [{'Term': '"Curvibacter"[Organism]', 'Field': 'Organism', 'Count': '28', 'Explode': 'Y'}, 'GROUP'], 'QueryTranslation': '"Curvibacter"[Organism]'}

In [45]:
handle = Entrez.elink(dbfrom="assembly",dbto="protein",id=record['IdList'],retmode="xml")
record = Entrez.read(handle)
handle.close()

In [46]:
record

[{'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['10501171']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['10449501']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['10109261']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['7805401']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['7793981']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['7787691']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['7781621']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['7743831']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': ['7742251']}, {'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'assembly', 'IdList': 

In [30]:
help(Entrez.elink)

Help on function elink in module Bio.Entrez:

elink(**keywds)
    Check for linked external articles and return a handle.
    
    ELink checks for the existence of an external or Related Articles link
    from a list of one or more primary IDs;  retrieves IDs and relevancy
    scores for links to Entrez databases or Related Articles; creates a
    hyperlink to the primary LinkOut provider for a specific ID and
    database, or lists LinkOut URLs and attributes for multiple IDs.
    
    See the online documentation for an explanation of the parameters:
    http://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ELink
    
    Return a handle to the results, by default in XML format.
    
    Raises an IOError exception if there's a network error.
    
    This example finds articles related to the Biopython application
    note's entry in the PubMed database:
    
    >>> from Bio import Entrez
    >>> Entrez.email = "Your.Name.Here@example.org"
    >>> pmid = "19304878"
    >>> handle =

In [37]:
handle = Entrez.elink(dbfrom="taxonomy",dbto="assembly",id=["281915"],linkname="taxonomy_assembly")
record = Entrez.read(handle)
handle.close()

In [38]:
record

[{'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'taxonomy', 'IdList': ['281915']}]

In [47]:
linked = [link["Id"] for link in record[0]["LinkSetDb"][0]["Link"]]

IndexError: list index out of range

In [16]:
len(linked)

1

In [17]:
linked

['15545462']

In [18]:
handle = Entrez.efetch(db="assembly",id=linked,retmode="xml")
record = Entrez.read(handle)
handle.close()

In [19]:
record

['15545462']

In [ ]:
queries = open(query_fasta_file, 'r')
lines = queries.readlines()
queries.close()
queries = []
for line in lines:
    if ">" in line:
        queries.append(line.split(" ")[0].split(".")[0].split(">")[1])

Entrez.email = 'lukas.becker@hhu.de'

# print(path_to_queries,queries)
handle = Entrez.efetch(db="protein", id=queries, retmode="xml")
record = Entrez.read(handle)
handle.close()

prot_to_pfam = {'QUERIES': [], 'PFAM': {}, 'TIGR': {}, 'REFSEQ': {}, 'CDD': {}, 'Definition':{}, 'Length':{}}
for i in range(len(record)):
    prot_to_pfam['QUERIES'].append(record[i]['GBSeq_locus'])
    prot_to_pfam['Definition'][record[i]['GBSeq_locus']] = record[i]['GBSeq_definition']
    prot_to_pfam['Length'][record[i]['GBSeq_locus']] = record[i]['GBSeq_length']
    # record[i]['GBSeq_locus']
    pfam = 'http://pfam.xfam.org/family/'
    jvci = 'https://www.ncbi.nlm.nih.gov/genome/annotation_prok/evidence/'
    cdd = 'https://www.ncbi.nlm.nih.gov/Structure/sparcle/archview.html?archid='
    refseq = 'https://www.ncbi.nlm.nih.gov/protein/' + str(record[i]['GBSeq_locus'])
    prot_to_pfam['REFSEQ'][record[i]['GBSeq_locus']] = refseq
    try:
        if "Domain architecture ID" in record[i]['GBSeq_comment']:
            cdd += record[i]['GBSeq_comment'].split('Domain architecture ID')[1].split(';')[0].strip()
            # print(cdd)
            prot_to_pfam['CDD'][record[i]['GBSeq_locus']] = cdd
        if "EMBL-EBI" in record[i]['GBSeq_comment']:
            pfam += record[i]['GBSeq_comment'].split("EMBL-EBI")[1].split("::")[1].split(";")[
                0].strip().rstrip()
            # prot_to_pfam.append((record[i]['GBSeq_locus'],pfam))
            prot_to_pfam['PFAM'][record[i]['GBSeq_locus']] = pfam
        if "TIGR" in record[i]['GBSeq_comment']:
            jvci += 'TIGR' + record[i]['GBSeq_comment'].split("TIGR")[1].split(";")[0].split(".")[
                0].strip().rstrip()
            # jvci += record[i]['GBSeq_comment'].split("JCVI")[1].split("::")[1].split(";")[0].strip().rstrip()
            # prot_to_pfam.append((record[i]['GBSeq_locus'],jvci))
            prot_to_pfam['TIGR'][record[i]['GBSeq_locus']] = jvci
            # prot_to_pfam.append((record[i]['GBSeq_locus'], 'no entry found'))
            # prot_to_pfam[record[i]['GBSeq_locus']] = "no entry in pfam database"
            # print(record[i]['GBSeq_locus'], pfam)
    except Exception as e:
        continue
    # print(record[i]['GBSeq_comment'])
    # print("#############")
    # json.dumps

In [5]:
prot_to_pfam['PFAM']

{'WP_087496569': 'http://pfam.xfam.org/family/PF00348.19',
 'WP_087496568': 'http://pfam.xfam.org/family/PF07589.13'}